# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [40]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [41]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [42]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [43]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [44]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.765903     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.813784     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [45]:
cof = model_all.coefficients
cof.print_rows(num_rows=19,num_columns=4 ) 

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [46]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate
import numpy as np # note this allows us to refer to numpy as np instead 

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [74]:
l1_list = np.logspace(1, 7, num=13)
print l1_list

[  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]


In [75]:
error_list = np.zeros(13)
for i in xrange(13):
    l1_penalty = float(l1_list[i:i+1])
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0.0, l1_penalty=l1_penalty,verbose = False)
    error = validation['price'] - model_all.predict(validation)
    weights = model_all['coefficients']['value']
    weights.apply(lambda x: abs(x))
    error_list[i] = (error*error).sum()  
    print l1_penalty , error_list[i]
print error_list

10.0 6.25766285142e+14
31.6227766017 6.25766285362e+14
100.0 6.25766286058e+14
316.227766017 6.25766288257e+14
1000.0 6.25766295212e+14
3162.27766017 6.25766317206e+14
10000.0 6.25766386761e+14
31622.7766017 6.25766606749e+14
100000.0 6.25767302792e+14
316227.766017 6.25769507644e+14
1000000.0 6.25776517727e+14
3162277.66017 6.25799062845e+14
10000000.0 6.25883719085e+14
[  6.25766285e+14   6.25766285e+14   6.25766286e+14   6.25766288e+14
   6.25766295e+14   6.25766317e+14   6.25766387e+14   6.25766607e+14
   6.25767303e+14   6.25769508e+14   6.25776518e+14   6.25799063e+14
   6.25883719e+14]


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [76]:
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0.0, l1_penalty=10.0,verbose = False)
    error = testing['price'] - model_all.predict(testing)
    RSS = (error*error).sum()
    print RSS

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [77]:
cof = model_all.coefficients
cof.print_rows(num_rows=19,num_columns=4 ) 

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [78]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [79]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [80]:
error_list = np.zeros(20)
for i in xrange(20):
    l1_penalty = float(l1_penalty_values[i:i+1])
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0.0, l1_penalty=l1_penalty,verbose = False)
    error = validation['price'] - model_all.predict(validation)
    error_list[i] = (error*error).sum()
    weights = model_all['coefficients']['value']
    n=len(weights)
    print weights
    non_zero = 0
    for j in xrange(n):
        if weights[j]==0:
            non_zero = non_zero + 1
    print l1_penalty , error_list[i], n - non_zero 
print error_list

[25090.917367212045, 7789.177061100557, 847.5596869428731, 25234.209194458625, 39.03944596356861, 1117.311895567563, -0.025686118239919547, 143.98899196977004, 20695.359239643883, 12466.69065029186, 568204.6445841154, 91066.94280879851, 6360.780926249804, 6139.212805647889, 43.035829924605636, 118.94587495435267, 9.040401654020453, 48.6154673092746]
100000000.0 6.27492659875e+14 18
[26746.66193663547, 7743.9790478498835, 822.358945250938, 25178.62593058695, 39.01071813525404, 1114.9107159232626, -0.01866307372278287, 142.51979784137168, 20545.86730469756, 12339.245250245609, 558930.2470718037, 90439.72185124883, 6288.009465538853, 6118.412320616763, 42.95755676813047, 117.99004260104356, 8.924085433195676, 46.56185624767167]
127427498.57 6.28210516771e+14 18
[28873.181016622373, 7691.0470756863215, 790.9175796843567, 25115.27853451593, 38.9820788132087, 1112.2394146479885, -0.02473736058080116, 140.94584475081732, 20365.265896852696, 12181.186257705755, 547143.180178905, 89651.69239158

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [81]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [82]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [83]:
error_list = np.zeros(20)
for i in xrange(20):
    l1_penalty = float(l1_penalty_values[i:i+1])
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0.0, l1_penalty=l1_penalty,verbose = False)
    error = validation['price'] - model_all.predict(validation)
    weights = model_all['coefficients']['value']
    n=len(weights)
    error_list[i] = (error*error).sum() 
    print weights
    non_zero = 0
    for j in xrange(n):
        if weights[j]==0:
            non_zero = non_zero + 1
    print l1_penalty , error_list[i], n - non_zero 
print error_list

[196100.93780562267, 2181.574321070672, 0.0, 17962.696661157817, 34.1424656512256, 789.3197890782325, 0.0, 0.0, 3665.9308175965184, 0.0, 0.0, 11333.841030833135, 0.0, 3578.900400438122, 32.74320137175841, 12.795381135944707, 0.0, 0.0]
2976351441.63 9.66925692362e+14 10
[198563.24621808337, 2067.015155558216, 0.0, 17810.387597798555, 34.02151030561595, 782.1823176948214, 0.0, 0.0, 3358.20330522381, 0.0, 0.0, 9876.73760811963, 0.0, 3528.255008870851, 32.53723292119236, 11.05072885745481, 0.0, 0.0]
3019316638.95 9.74019450085e+14 10
[201025.56040541275, 1952.476489609666, 0.0, 17658.089496535536, 33.9005653546246, 775.0454285495273, 0.0, 0.0, 3050.4721940384206, 0.0, 0.0, 8418.987059021993, 0.0, 3477.6121661732705, 32.33126559851515, 9.306267213386917, 0.0, 0.0]
3062281836.27 9.81188367942e+14 10
[203443.60100002843, 1825.7886426921766, 0.0, 17487.492351087196, 33.76199379582765, 766.9631650719964, 0.0, 0.0, 2716.8076217860184, 0.0, 0.0, 6944.366160803107, 0.0, 3421.350352523167, 32.10532

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [85]:

l1_penalty = 3448968612.16
model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0.0, l1_penalty=l1_penalty,verbose = False)
cof = model_all.coefficients
cof.print_rows(num_rows=19,num_columns=4 ) 

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |